In [1]:
import numpy as np
from scipy.stats import bernoulli

#K groups,N arms/group, delta=confidence of success
# Bernoulli reward disrtibution
#reward distribution example 1:[[0.1,0.2,0.3,0.2],[0.6,0.6,0.6,0.6],[0.9,0.8,0.7,0.6]]
#reward distribution example 2:[[0.1,0.2,0.3,0.2],[0.1,0.1,0.1,0.1],[0.9,0.8,0.7,0.6]]--easier to distinguish

K=3;N=4;delta=1;min_threshold=0.15;
reward_distribution=np.array([[0.1,0.2,0.3,0.2],[0.3,0.4,0.3,0.3],[0.9,0.8,0.7,0.6]]);
pulls=np.ones((K,N));
active_groups=np.ones(K);
actual_reward_received=np.zeros((K,N));
t=1;
lcb_arms=np.zeros((K,N));ucb_arms=np.zeros((K,N));
lcb_group=np.zeros(K);ucb_group=np.zeros(K);


def eliminate_arms(active_groups,lcb_group,ucb_group):
    lcb_max=np.max(lcb_group);
    state_groups1=(ucb_group>=lcb_max);
    state_groups2=np.array(list(map(sum, (ucb_arms>=min_threshold))))>=1;
    active_groups*=state_groups1;
    return active_groups
    
def calculate_bounds(lcb_group,ucb_group,lcb_arms,ucb_arms):
    lcb_arms=actual_reward_received/pulls-np.sqrt(np.log(4*t*t/delta)/(2*pulls));
    ucb_arms=actual_reward_received/pulls+np.sqrt(np.log(4*t*t/delta)/(2*pulls));
    lcb_group=lcb_arms.sum(axis=1);
    ucb_group=ucb_arms.sum(axis=1);
    return lcb_arms,ucb_arms,lcb_group,ucb_group;

def eliminate_minviolating_arms(active_groups,lcb_arms,ucb_arms):
    for group in range(K):
        if active_groups[group]==1: 
            if np.prod(ucb_arms[group]>=min_threshold)==0 :
                #print("ucb=",ucb_arms[group])
                active_groups[group]=0;
            if np.prod(lcb_arms[group]>=min_threshold)==1:
                active_groups[group]=2;  #2 =dormant group
    return active_groups      
    
    
while np.bincount(np.int32(active_groups))[1]!=0 and t<=500000:
    for i in range(len(active_groups)):
        if active_groups[i]==1:
            
            for index in range(len(reward_distribution[i])):
                pulls[i][index]+=1;
                t=t+1;
                actual_reward_received[i][index]+=bernoulli.rvs(p=reward_distribution[i][index]);
    lcb_arms,ucb_arms,lcb_group,ucb_group=calculate_bounds(lcb_group,ucb_group,lcb_arms,ucb_arms);
    active_groups=eliminate_minviolating_arms(active_groups,lcb_arms,ucb_arms);
    #print(active_groups,"\n",lcb_arms,"\n",ucb_arms)
    
print(active_groups,"\n",lcb_arms,"\n",ucb_arms)

while np.sum((active_groups>=1))>1 and t<=500000 :
    for i in range(len(active_groups)):
        if active_groups[i]>=1:
            
            for index in range(len(reward_distribution[i])):
                pulls[i][index]+=1;
                t=t+1;
                actual_reward_received[i][index]+=bernoulli.rvs(p=reward_distribution[i][index]);

    
    lcb_arms,ucb_arms,lcb_group,ucb_group=calculate_bounds(lcb_group,ucb_group,lcb_arms,ucb_arms);
    active_groups=eliminate_arms(active_groups,lcb_group,ucb_group);
    #print(lcb_group,ucb_group,"\n",pulls)

print("Stopping Time :",t)
print("Active arm :",active_groups)


[0. 2. 2.] 
 [[0.03533853 0.14236728 0.24715961 0.15163246]
 [0.17305165 0.23365771 0.14072841 0.14476882]
 [0.21731351 0.06346735 0.21731351 0.02500582]] 
 [[0.14996499 0.25699374 0.36178608 0.26625892]
 [0.46129179 0.52189785 0.42896856 0.43300896]
 [1.47499418 1.32114803 1.47499418 1.28268649]]
Stopping Time : 15249
Active arm : [0. 0. 2.]


# Observations
##  1)if min_threshold==mean of one the arms then,it will take infinite time to discard that arm.
##  2) we have to separate the process in two stages as,if the group confidence bounds separates first then the higher group might have a min_violating arm which is undesirable
##  3)Iterate until lcb of all arms goes above min_threshold(mark as dormant) or 
##   ucb of any arm goes below min_threshold(mark as inactive) 
##  4) Only dormant arms proceed to successive elimination stage.